In [2]:
#code for making artificial dataset
import random
def random_string(length,alphabet_list):
    rand_str = ''.join(random.choice(alphabet_list) for i in range(length))
    return rand_str

def perturb(seed,alphabet_list,p=0.5):
    seq=''
    for c in seed:
        if random.random() < p: c = random.choice(alphabet_list)
        seq += c
    return seq

def make_artificial_dataset(alphabet='ACGT', motives=None, motif_length=6, 
                            sequence_length=100, n_sequences=1000, n_motives=2, p=0.2,
                           random_state=1):
    random.seed(random_state)
    alphabet_list=[c for c in alphabet]
    
    if motives is None:
        motives=[]
        for i in range(n_motives):
            motives.append(random_string(motif_length,alphabet_list))
    else:
        motif_length = len(motives[0])
        n_motives = len(motives)
    
    sequence_length = sequence_length / len(motives)
    flanking_length = (sequence_length - motif_length ) / 2
    n_seq_per_motif = n_sequences

    counter=0
    seqs=[]
    for i in range(n_seq_per_motif):
        total_seq = ''
        total_binary_seq=''
        for j in range(n_motives):
            left_flanking = random_string(flanking_length,alphabet_list)
            right_flanking = random_string(flanking_length,alphabet_list)
            noisy_motif = perturb(motives[j],alphabet_list,p)
            seq = left_flanking + noisy_motif + right_flanking
            total_seq += seq
        seqs.append(('>ID%d'%counter,total_seq))
        counter += 1
    binary_skeleton = '0' * flanking_length + '1' * motif_length + '0' * flanking_length
    binary_seq = binary_skeleton * n_motives
    return motives, seqs, binary_seq

In [3]:
from sklearn.cluster import KMeans
from utilities import Weblogo, MuscleAlignWrapper
from eden_wrapper import EdenWrapper
from meme_wrapper import Meme

In [4]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [5]:
def run_tool(motif_finder,seqs, n_motives, min_motif_len, max_motif_len):
    if motif_finder=='meme':
        with open('seqs.fa','w') as f_train:
            for seq in seqs:
                f_train.write('>' + seq[0] + ' \n')
                f_train.write(seq[1] + '\n')

        tool =  Meme(alphabet='dna',
                     minw=min_motif_len,
                     maxw=max_motif_len,
                     nmotifs=n_motives,
                     maxsize=100000)
        tool.fit('seqs.fa')
    else:
        km = KMeans(n_clusters=n_motives)
        tool = EdenWrapper(alphabet='dna', 
                           complexity=5, 
                           nbits=14, 
                           negative_ratio=3,
                           min_subarray_size=min_motif_len, 
                           max_subarray_size=max_motif_len,
                           clustering_algorithm=km)
        tool.fit(seqs)
    return tool

In [6]:
def score_seqs(seqs, n_motives, scoring_criteria, tool):
    scores = []
    for j in range(len(seqs)):
        seq_scr = []
        for k in range(n_motives):
            if scoring_criteria=='pwm':
                scr=tool.score_pwm(motif_num=k+1, seq=seqs[j][1], zero_padding=True)
            else: # scoring_criteria=='hmm'
                scr=tool.score_pwm(motif_num=k+1, seq=seqs[j][1], zero_padding=True)
            seq_scr.append(scr)

        # taking average over all motives for a sequence
        x = np.array(seq_scr[0])
        for k in range(1, n_motives):
            x = np.vstack((x, seq_scr[k]))
        seq_scr = list(np.mean(x, axis=0))
        scores.append(seq_scr)
    return scores
#investigate max or mean

In [7]:
import numpy as np
from sklearn.metrics import roc_auc_score

def evaluate(motif_finder='meme', # ['meme','eden']
             scoring_criteria='pwm', # ['meme','hmm']
             motives=None,
             motif_length=6,
             n_motives=4,
             sequence_length=100,
             n_sequences=50,
             p=0.2,
             random_state=8):

    motives, seqs, binary_seq = make_artificial_dataset(alphabet='ACGT',
                                                        motives=motives,
                                                        sequence_length=sequence_length,
                                                        n_sequences=n_sequences,
                                                        motif_length=motif_length,
                                                        n_motives=n_motives,
                                                        p=p,
                                                        random_state=random_state)
    tool = run_tool(motif_finder=motif_finder,
                    seqs=seqs,
                    n_motives=n_motives,
                    min_motif_len=max(2, len(motives[0])-2),
                    max_motif_len=len(motives[0])+2)
    
    scores = score_seqs(seqs=seqs,
                        n_motives=n_motives,
                        scoring_criteria=scoring_criteria,
                        tool=tool)
    roc_scores = []
    true_score = [float(int(x)) for x in binary_seq]
    for score in scores:
        roc_scores.append(roc_auc_score(true_score, score))
    avg_roc = np.average(roc_scores)
    max_roc = max(roc_scores)
    return avg_roc, max_roc, roc_scores

<h3>Experiment 1: Varying number of motives</h3>

In [18]:
%%time
avg_roc_list=[]
max_roc_list=[]
roc_scores_list=[]

parameter = range(4,9)

for i in parameter:
    avg_roc, max_roc, roc_scores = evaluate(motif_finder='meme', # ['meme','eden']
                                            scoring_criteria='pwm', # ['pwm','hmm']
                                            motif_length=15,
                                            n_motives=i,
                                            sequence_length=300,
                                            n_sequences=200,
                                            p=0.01)
    avg_roc_list.append(avg_roc)
    max_roc_list.append(max_roc)
    roc_scores_list.append(roc_scores)

INFO:meme_wrapper:The output directory 'meme_out' already exists.
Its contents will be overwritten.
Initializing the motif probability tables for 2 to 200 sites...
nsites = 200
Done initializing.
SEEDS: highwater mark: seq 199 pos 300

seqs=   200, min= 300, max=  300, total=    60000

motif=1
SEED WIDTHS: 13 17
em: w=  17, psites= 200, iter=   0 
motif=2
SEED WIDTHS: 13 17
em: w=  17, psites= 200, iter=   0 
motif=3
SEED WIDTHS: 13 17
em: w=  17, psites= 200, iter=   0 
motif=4
SEED WIDTHS: 13 17
em: w=  17, psites= 200, iter=   0 

INFO:meme_wrapper:The output directory 'meme_out' already exists.
Its contents will be overwritten.
Initializing the motif probability tables for 2 to 200 sites...
nsites = 200
Done initializing.
SEEDS: highwater mark: seq 199 pos 295

seqs=   200, min= 295, max=  295, total=    59000

motif=1
SEED WIDTHS: 13 17
em: w=  17, psites= 200, iter=   0 
motif=2
SEED WIDTHS: 13 17
em: w=  17, psites= 200, iter=   0 
motif=3
SEED WIDTHS: 13 17
em: w=  17, psites= 

CPU times: user 1min 5s, sys: 396 ms, total: 1min 5s
Wall time: 14min 53s


In [19]:
print "Number of Motives \t Avg-ROC \t Max-ROC"
for i,r in enumerate(avg_roc_list):
    print "%d \t\t %f \t %f"%(parameter[i],r, max_roc_list[i])

Number of Motives 	 Avg-ROC 	 Max-ROC
4 		 0.533333 	 0.533333
5 		 0.533333 	 0.533333
6 		 0.533333 	 0.533333
7 		 0.533333 	 0.533333
8 		 0.533354 	 0.537500


<h3>Experiment 2: Varying Motif Length</h3>

In [20]:
%%time
avg_roc_list=[]
max_roc_list=[]
roc_scores_list=[]

parameter = range(10,16)

for i in parameter:
    avg_roc, max_roc, roc_scores = evaluate(motif_finder='meme', # ['meme','eden']
                          scoring_criteria='pwm', # ['pwm','hmm']
                          motif_length=i,
                          n_motives=5,
                          sequence_length=150,
                          n_sequences=100,
                          p=0.01)
    avg_roc_list.append(avg_roc)
    max_roc_list.append(max_roc)
    roc_scores_list.append(roc_scores)

INFO:meme_wrapper:The output directory 'meme_out' already exists.
Its contents will be overwritten.
Initializing the motif probability tables for 2 to 100 sites...
nsites = 100
Done initializing.
SEEDS: highwater mark: seq 99 pos 150

seqs=   100, min= 150, max=  150, total=    15000

motif=1
SEED WIDTHS: 8 11 12
em: w=  12, psites= 100, iter=   0 
motif=2
SEED WIDTHS: 8 11 12
em: w=  12, psites= 100, iter=   0 
motif=3
SEED WIDTHS: 8 11 12
em: w=  12, psites= 100, iter=   0 
motif=4
SEED WIDTHS: 8 11 12
em: w=  12, psites= 100, iter=   0 
motif=5
SEED WIDTHS: 8 11 12
em: w=  12, psites= 100, iter=   0 

INFO:meme_wrapper:The output directory 'meme_out' already exists.
Its contents will be overwritten.
Initializing the motif probability tables for 2 to 100 sites...
nsites = 100
Done initializing.
SEEDS: highwater mark: seq 99 pos 145

seqs=   100, min= 145, max=  145, total=    14500

motif=1
SEED WIDTHS: 9 12 13
em: w=  13, psites= 100, iter=   0 
motif=2
SEED WIDTHS: 9 12 13
em: w=  

CPU times: user 12.7 s, sys: 120 ms, total: 12.9 s
Wall time: 1min 49s


In [21]:
print "Motif Length \t Avg-ROC \t Max-ROC"
for i,r in enumerate(avg_roc_list):
    print "%d \t\t %f \t %f"%(parameter[i],r,max_roc_list[i])

Motif Length 	 Avg-ROC 	 Max-ROC
10 		 0.549964 	 0.560000
11 		 0.545545 	 0.554545
12 		 0.541667 	 0.541667
13 		 0.538462 	 0.538462
14 		 0.535714 	 0.535714
15 		 0.533333 	 0.533333


<h3>Experiment 3: Fixed Motives with Varying Sequence Length</h3>

In [19]:
%%time

semi_len=10
motives=['A'*semi_len+'C'*semi_len,
         'C'*semi_len+'A'*semi_len,
         'A'*semi_len+'T'*semi_len,
         'T'*semi_len+'A'*semi_len,
         'A'*semi_len+'G'*semi_len,
         'G'*semi_len+'A'*semi_len,
         'G'*semi_len+'C'*semi_len,
         'C'*semi_len+'G'*semi_len,]

avg_roc_list=[]
max_roc_list=[]
roc_scores_list=[]

parameter = range(250,510,50)

for i in parameter:
    avg_roc, max_roc, roc_scores = evaluate(motif_finder='meme', # ['meme','eden']
                                            scoring_criteria='pwm', # ['pwm','hmm']
                                            motives=motives,
                                            sequence_length=i,
                                            n_sequences=200,
                                            p=0.01)
    avg_roc_list.append(avg_roc)
    max_roc_list.append(max_roc)
    roc_scores_list.append(roc_scores)

INFO:meme_wrapper:The output directory 'meme_out' already exists.
Its contents will be overwritten.
Initializing the motif probability tables for 2 to 200 sites...
nsites = 200
Done initializing.
SEEDS: highwater mark: seq 199 pos 240

seqs=   200, min= 240, max=  240, total=    48000

motif=1
SEED WIDTHS: 18 22
em: w=  22, psites= 200, iter=  10 
motif=2
SEED WIDTHS: 18 22
em: w=  22, psites= 200, iter=  30 
motif=3
SEED WIDTHS: 18 22
em: w=  22, psites= 200, iter=  10 
motif=4
SEED WIDTHS: 18 22
em: w=  22, psites= 200, iter=  40 

INFO:meme_wrapper:The output directory 'meme_out' already exists.
Its contents will be overwritten.
Initializing the motif probability tables for 2 to 200 sites...
nsites = 200
Done initializing.
SEEDS: highwater mark: seq 199 pos 288

seqs=   200, min= 288, max=  288, total=    57600

motif=1
SEED WIDTHS: 18 22
em: w=  22, psites= 200, iter=  10 
motif=2
SEED WIDTHS: 18 22
em: w=  22, psites= 200, iter=  10 
motif=3
SEED WIDTHS: 18 22
em: w=  22, psites= 

CPU times: user 1min 26s, sys: 496 ms, total: 1min 27s
Wall time: 21min 1s


In [20]:
print "Sequence Length \t Avg-ROC \t Max-ROC"
for i,r in enumerate(avg_roc_list):
    print "%d \t\t %f \t %f"%(parameter[i],r,max_roc_list[i])

Sequence Length 	 Avg-ROC 	 Max-ROC
250 		 0.488609 	 0.496953
300 		 0.515568 	 0.518750
350 		 0.511524 	 0.518750
400 		 0.516370 	 0.525000
450 		 0.510479 	 0.518750
500 		 0.510162 	 0.514183


<h3>Experiment 4: Varying Motif Length (Fixed Motives)</h3>

In [12]:
%%time

parameter = range(10,31,5)

avg_roc_list=[]
max_roc_list=[]
roc_scores_list=[]

for i in parameter:
    motives=['A'*i,
             'C'*i,
             'A'*i,
             'C'*i]
    
    avg_roc, max_roc, roc_scores = evaluate(motif_finder='eden', # ['meme','eden']
                                            scoring_criteria='hmm', # ['pwm','hmm']
                                            motives=motives,
                                            sequence_length=300,
                                            n_sequences=200,
                                            p=0.001)
    avg_roc_list.append(avg_roc)
    max_roc_list.append(max_roc)
    roc_scores_list.append(roc_scores)

DEBUG:eden.util:Positive data: Instances: 200 ; Features: 16385 with an avg of 8310 features per instance
DEBUG:eden.util:Negative data: Instances: 600 ; Features: 16385 with an avg of 8438 features per instance
DEBUG:eden.util:Elapsed time: 78.0 secs
INFO:eden.motif:model induction: 200 positive instances 80 s
INFO:eden.motif:motives extraction: 708 motives in 26s
INFO:eden.motif:motives clustering: 4 clusters in 0s
INFO:eden.motif:after filtering: 66 motives 4 clusters in 0s
INFO:eden.motif:motif model construction in 0s
INFO:eden.motif:updated motif counts in 0s
DEBUG:eden.util:Positive data: Instances: 200 ; Features: 16385 with an avg of 8074 features per instance
DEBUG:eden.util:Negative data: Instances: 600 ; Features: 16385 with an avg of 8382 features per instance
DEBUG:eden.util:Elapsed time: 91.8 secs
INFO:eden.motif:model induction: 200 positive instances 94 s
INFO:eden.motif:motives extraction: 674 motives in 22s
INFO:eden.motif:motives clustering: 4 clusters in 0s
INFO:ed

CPU times: user 6min 28s, sys: 1min 1s, total: 7min 29s
Wall time: 10min 55s


In [13]:
print "Motif Length \t Avg-ROC \t Max-ROC"
for i,r in enumerate(avg_roc_list):
    print "%d \t\t %f \t %f"%(parameter[i],r,max_roc_list[i])

Motif Length 	 Avg-ROC 	 Max-ROC
10 		 0.512642 	 0.591162
15 		 0.524207 	 0.581146
20 		 0.494075 	 0.523553
25 		 0.496301 	 0.522750
30 		 0.498071 	 0.517874


<h3>Experiment 5: Varying Motif Length (Fixed Motives) with MEME</h3>

In [11]:
%%time

parameter = range(10,16,5)

avg_roc_list=[]
max_roc_list=[]
roc_scores_list=[]

for i in parameter:
    motives=['A'*i,
             'C'*i,
             'A'*i,
             'C'*i]
    
    avg_roc, max_roc, roc_scores = evaluate(motif_finder='meme', # ['meme','eden']
                                            scoring_criteria='hmm', # ['pwm','hmm']
                                            motives=motives,
                                            sequence_length=300,
                                            n_sequences=200,
                                            p=0.001)
    avg_roc_list.append(avg_roc)
    max_roc_list.append(max_roc)
    roc_scores_list.append(roc_scores)

INFO:meme_wrapper:The output directory 'meme_out' already exists.
Its contents will be overwritten.
Initializing the motif probability tables for 2 to 200 sites...
nsites = 200
Done initializing.
SEEDS: highwater mark: seq 199 pos 296

seqs=   200, min= 296, max=  296, total=    59200

motif=1
SEED WIDTHS: 8 11 12
em: w=  12, psites= 200, iter=  40 
motif=2
SEED WIDTHS: 8 11 12
em: w=  12, psites= 200, iter=  20 
motif=3
SEED WIDTHS: 8 11 12
em: w=  12, psites= 200, iter=  40 
motif=4
SEED WIDTHS: 8 11 12
em: w=  12, psites= 200, iter=  40 

INFO:meme_wrapper:The output directory 'meme_out' already exists.
Its contents will be overwritten.
Initializing the motif probability tables for 2 to 200 sites...
nsites = 200
Done initializing.
SEEDS: highwater mark: seq 199 pos 300

seqs=   200, min= 300, max=  300, total=    60000

motif=1
SEED WIDTHS: 13 17
em: w=  17, psites= 200, iter=  20 
motif=2
SEED WIDTHS: 13 17
em: w=  17, psites= 200, iter=  30 
motif=3
SEED WIDTHS: 13 17
em: w=  17, 

KeyboardInterrupt: 

In [ ]:
print "Motif Length \t Avg-ROC \t Max-ROC"
for i,r in enumerate(avg_roc_list):
    print "%d \t\t %f \t %f"%(parameter[i],r,max_roc_list[i])